![Vidi_Waterflux_Banner](https://raw.githubusercontent.com/ITC-Water-Resources/Vidi-waterflux-merch/refs/heads/main/jupyter/Vidi_Waterflux_Banner.png)
*Roelof Rietbroek, Sedigheh Karimi, Amin Shakya EGU 2025*

# Compute cumulative P,ET storage changes and climatology

In [2]:
import os
import geopandas as gpd
import shxarray
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from scipy.integrate import cumulative_trapezoid
from common.config import read_config

In [3]:
conf=read_config()
datadir=conf["dataroot"]
p_etbasinfile=os.path.join(datadir,conf['ncp-et'])
p_etbasin_comb_file=os.path.join(datadir,conf['ncp-et_comb'])


In [4]:
ds_flux=xr.open_dataset(p_etbasinfile).rename(valid_time='time')

ds_flux

<xarray.Dataset> Size: 575kB
Dimensions:     (time: 276, basin: 124)
Coordinates:
    number      int64 8B ...
  * time        (time) datetime64[ns] 2kB 2002-04-01T06:00:00 ... 2025-03-01T...
    expver      (time) <U4 4kB ...
  * basin       (basin) <U39 19kB 'SETIT' ... 'TIBETAN PLATEAU'
    basin_area  (basin) float64 992B ...
Data variables:
    avg_ie      (basin, time) float64 274kB ...
    avg_tprate  (basin, time) float64 274kB ...

## Compute cumulative monthly precipitation and evapotranspiration

In [5]:
# cumulative integrate fluxes

#compute mean fluxes
ds_flux['p_mean']=ds_flux.avg_tprate.mean('time')
ds_flux['et_mean']=ds_flux.avg_ie.mean('time')

#cumulative integrations after removing the mean
#delta time since the first epoch as secvonds
deltat=(ds_flux.time-ds_flux.time[0]).astype('timedelta64[s]').astype(int)

delta_p=ds_flux.avg_tprate-ds_flux['p_mean']
ds_flux['p_cuma']=(['basin','time'],cumulative_trapezoid(y=delta_p.data,x=deltat,initial=0))

delta_et=ds_flux.avg_ie-ds_flux['et_mean']
ds_flux['et_cuma']=(['basin','time'],cumulative_trapezoid(y=delta_et.data,x=deltat,initial=0))




In [6]:
#alos compute monthly climatologies
ds_flux['p_clima']=ds_flux.avg_tprate.groupby('time.month').mean()
ds_flux['et_clima']=ds_flux.avg_ie.groupby('time.month').mean()

ds_flux

<xarray.Dataset> Size: 1MB
Dimensions:     (time: 276, basin: 124, month: 12)
Coordinates:
    number      int64 8B ...
  * time        (time) datetime64[ns] 2kB 2002-04-01T06:00:00 ... 2025-03-01T...
    expver      (time) <U4 4kB ...
  * basin       (basin) <U39 19kB 'SETIT' ... 'TIBETAN PLATEAU'
    basin_area  (basin) float64 992B ...
  * month       (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    avg_ie      (basin, time) float64 274kB -1.593e+03 -1.513e+03 ... -2.746e+03
    avg_tprate  (basin, time) float64 274kB 315.8 158.0 ... 1.665e+03 2.051e+03
    p_mean      (basin) float64 992B 906.8 3.452e+03 ... 8.558e+03 8.724e+03
    et_mean     (basin) float64 992B -1.869e+03 -4.779e+03 ... -7.234e+03
    p_cuma      (basin, time) float64 274kB 0.0 -1.736e+09 ... 4.867e+10
    et_cuma     (basin, time) float64 274kB 0.0 8.174e+08 ... -3.021e+10
    p_clima     (basin, month) float64 12kB 1.035e+03 654.3 ... 1.048e+03 796.0
    et_clima    (basin, month) float64 12kB -1.85e+03 -1.723e+03 ... -1.292e+03

In [7]:


#Save combined results to file
ds_flux.to_netcdf(p_etbasin_comb_file)
